In [1]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv("F:\SE\SEM 2\PBL\spam - Email Dataset.csv")
df.head()

,Unnamed: 0,Unnamed: 1
0,Category,Message
1,ham,"Go until jurong point, crazy.. Available only ..."
2,ham,Ok lar... Joking wif u oni...
3,spam,Free entry in 2 a wkly comp to win FA Cup fina...
4,ham,U dun say so early hor... U c already then say...


In [29]:
df.Category.value_counts()

ham     2111
spam     341
Name: Category, dtype: int64

In [30]:
df['spam'] = df['Category'].apply(lambda x: 1 if x =='spam' else 0)

In [31]:
df.shape

(2452, 3)

In [32]:
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


# Train test split

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Message, df.spam, test_size=0.2)

In [34]:
X_train.shape

(1961,)

In [35]:
X_test.shape

(491,)

In [36]:
type(X_train)

pandas.core.series.Series

In [37]:
X_train[:4]

1717    Sorry about earlier. Putting out fires.Are you...
242         PLEASSSSSSSEEEEEE TEL ME V AVENT DONE SPORTSx
244     Although i told u dat i'm into baig face watch...
1418    No wonder... Cos i dun rem seeing a silver car...
Name: Message, dtype: object

In [38]:
type(y_train)

pandas.core.series.Series

In [39]:
y_train[:4]

1717    0
242     0
244     0
1418    0
Name: spam, dtype: int64

In [40]:
type(X_train.values)

numpy.ndarray

# Create bag of words representation using CountVectorizer

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

X_train_cv = v.fit_transform(X_train.values)
X_train_cv

<1961x5017 sparse matrix of type '<class 'numpy.int64'>'
	with 26572 stored elements in Compressed Sparse Row format>

In [42]:
X_train_cv.toarray()[:2][0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [43]:
X_train_cv.shape

(1961, 5017)

In [44]:
X_train_np = X_train_cv.toarray()
X_train_np[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [45]:
np.where(X_train_np[0]!=0)

(array([ 469,  495,  521,  661,  675, 1519, 1606, 1842, 2137, 2638, 2692,
        3198, 3223, 3539, 4077, 4311, 4470, 4988], dtype=int64),)

# Train the naive bayes model

In [46]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_cv, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [47]:
X_test_cv = v.transform(X_test)

# Evaluate Performance

In [48]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_cv)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       421
           1       1.00      0.89      0.94        70

    accuracy                           0.98       491
   macro avg       0.99      0.94      0.96       491
weighted avg       0.98      0.98      0.98       491



In [50]:
emails = [
    ' NPTEL+: Upcoming Workshops - Hurry up and Register today!',
    'Summer Sale Is Live'
]

emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1], dtype=int64)

# Train the model using sklearn pipeline and reduce number of lines of code

In [51]:
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [25]:
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('nb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [26]:
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       418
           1       0.97      0.88      0.92        73

    accuracy                           0.98       491
   macro avg       0.97      0.94      0.95       491
weighted avg       0.98      0.98      0.98       491

